# Генерация данных для витрины ИНИД

Сгенерируем данные по доходам чиновников, [выложенные в открытый доступ на платформе ИНИД](https://data-in.ru/data-catalog/datasets/150/).

In [1]:
import pandas as pd
import os

from decide import *

## Парсинг деклараций

Проведем скачивание и парсинг всех деклараций, информация о которых находится в `config/config.csv`. Этот процесс может занять некоторое время, особенно если сайты министерств медленно отвечают на запросы.

In [2]:
if not os.path.exists('data/'):
    relevant_columns = ['Код министерства', 
                        'Наименование министерства', 
                        'Год',
                        'Ссылка на раздел сайта с декларациями',
                        'Ссылка на файл с декларациями']

    config = load_config()[relevant_columns]

    valid_dir('data/')

    def parse_declaration(row):
        print('Parsing with {}, {}, {}'.format(row[0], row[1], row[2]))
        decide(row[0], row[1], row[2], config)
    
    config.drop_duplicates(subset=['Код министерства', 'Год']).apply(parse_declaration, axis=1)

## Соберем результат парсинга всех деклараций в одну таблицу

In [3]:
df = pd.DataFrame()

csv_path = 'data/csv_files/'
csv_files = [os.path.join(csv_path, fn) for fn in os.listdir(csv_path)]

df = pd.concat([pd.read_csv(cf, sep=';') for cf in csv_files], axis=0)
df.drop_duplicates(subset='id', inplace=True)

## Нормализация данных

В таблице, выложенной на сайт ИНИД, – данные только о доходах (не об имуществе) самих чиновников, названия должностей сгруппированы, вручную заполнены пропуски, а суммы приведены к постоянным ценам 2019 года. 

In [4]:
from src.inid.yaspeller import use_yandexspeller
from src.inid.normalizer import *

header = [  
    'id',
    'name',
    'position',
    'position_corrected',
    'income',
    'income_add',
    'income_diff',
    'income_desc',
    'state_name',
    'state_name_long',
    'year'
]

clerks = pd.DataFrame(columns=header)

In [ ]:
clerks['id'] = df['id'].values.tolist()
clerks['name'] = df['name'].values.tolist()
clerks['position'] = df['position'].values.tolist()
clerks['income'] = df['money'].apply(normalize_money).values.tolist()
clerks['income_add'] = df['money_desc'].apply(normalize_money_add).values.tolist()
clerks['income_diff'] = df.apply(calc_money_diff, axis=1).values.tolist()
clerks['income_desc'] = df['money_desc'].apply(normalize_money_desc).values.tolist()
clerks['state_name'] = df['state_name'].apply(shorten_name).values.tolist()
clerks['state_name_long'] = df['state_name'].values.tolist()
clerks['year'] = df['year'].values.tolist()

positions = set(name for name,group in df.groupby('position'))
positions_corrected = {p: use_yandexspeller(p) for p in positions}
clerks['position_corrected'] = clerks['position'].map(positions_corrected)

Удаляем министерства, в которых применяется другая система должностей

In [ ]:
clerks = clerks[~clerks['state_name'].isin(['МВД России', 'МИД России'])]

Группируем должности

In [ ]:
clerks['position_standard'] = clerks['position_corrected'].apply(find_category)
clerks['category'] = clerks['position_standard'].map(category_dict)
clerks['group'] = clerks['position_standard'].map(group_dict)
clerks['category'] = clerks['category'].replace(r'^\s*$', None, regex=True)
clerks['group'] = clerks['group'].replace(r'^\s*$', None, regex=True)

## Создаем колонки с доходом в постояннных ценах 2019 года

In [ ]:
from decimal import Decimal

dict = {
    2010: 1.756666993, 
    2011: 1.655671058, 
    2012: 1.553599567, 
    2013: 1.459189975, 
    2014: 1.310453503, 
    2015: 1.160617751,
    2016: 1.101259846,
    2017: 1.07429504, 
    2018: 1.0304, 
    2019: 1.0
}

clerks['coef'] = clerks['year'].map(dict)
clerks['income_month_const'] = round((clerks['income'].astype(float) / 12) * clerks['coef'], 2)
clerks['income_diff_month_const'] = round((clerks['income'].astype(float) / 12) * clerks['coef'], 2)

In [ ]:
header = ['name', 
    'position', 
    'position_standard', 
    'category', 
    'group', 
    'state_name', 
    'state_name_long', 
    'year', 
    'income', 
    'income_desc', 
    'income_add',
    'income_diff', 
    'income_month_const', 
    'income_diff_month_const'
]

clerks = clerks[header]

clerks.to_csv('data/clerks.csv', sep=';', index=False)
clerks.to_excel('data/income.xlsx', index = True)

In [ ]:
display(clerks)